# From text to Knowledge Graph
In this notebook we will see how we can transform a simple sentence into a Knowledge Graph using simple steps.

## Natural Language Processing

In [275]:
import spacy
import spacy_dbpedia_spotlight
from spacy.matcher import Matcher 
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('dbpedia_spotlight')
doc = nlp("Joe Biden married Jill Tracy Jacobs, who was educated at the University of Delaware")

# Tokenization
print([token.text for token in doc], sep=' ')

['Joe', 'Biden', 'married', 'Jill', 'Tracy', 'Jacobs', ',', 'who', 'was', 'educated', 'at', 'the', 'University', 'of', 'Delaware']


In [33]:
# Lemmatisation
print([token.lemma_ for token in doc], sep=' ')

['Joe', 'Biden', 'marry', 'Jill', 'Tracy', 'Jacobs', ',', 'who', 'be', 'educate', 'at', 'the', 'University', 'of', 'Delaware']


In [222]:
print([token.lemma_ for token in doc], sep=' ')

['DBPEDIA_ENT', 'DBPEDIA_ENT', '', 'DBPEDIA_ENT', 'DBPEDIA_ENT', '', '', '', '', '', '', '', '', '', 'DBPEDIA_ENT']


In [35]:
# Part-of-speech Tagging
print([(token.lemma_, token.pos_) for token in doc], sep=' ')

[('Joe', 'PROPN'), ('Biden', 'PROPN'), ('marry', 'VERB'), ('Jill', 'PROPN'), ('Tracy', 'PROPN'), ('Jacobs', 'PROPN'), (',', 'PUNCT'), ('who', 'PRON'), ('be', 'AUX'), ('educate', 'VERB'), ('at', 'ADP'), ('the', 'DET'), ('University', 'PROPN'), ('of', 'ADP'), ('Delaware', 'PROPN')]


In [39]:
# Dependency Parsing
print([(token.lemma_, token.dep_) for token in doc], sep=' ')

[('Joe', 'compound'), ('Biden', 'nsubj'), ('marry', 'ROOT'), ('Jill', 'compound'), ('Tracy', 'compound'), ('Jacobs', 'dobj'), (',', 'punct'), ('who', 'nsubjpass'), ('be', 'auxpass'), ('educate', 'relcl'), ('at', 'prep'), ('the', 'det'), ('University', 'pobj'), ('of', 'prep'), ('Delaware', 'pobj')]


In [40]:
from spacy import displacy
displacy.render(doc, style="dep")

## Named Entity Recognition (NER)
Named Entity Recognition (NER) identifies mentions of named entities in a text, typically targetting mentions of people, organisations, locations, and potentially other types.

In [38]:
print([(ent.text, ent.label_) for ent in doc.ents], sep = ' ')

[('Joe Biden', 'PERSON'), ('Jill Tracy Jacobs', 'PERSON'), ('the University of Delaware', 'ORG')]


In [272]:
ents = [ent for ent in doc.ents]

In [284]:
ents_t = [ent.text for ent in doc.ents]

In [279]:
print("Before:", [token.text for token in doc])

with doc.retokenize() as retokenizer:
    for i in range(len(ents)):
        retokenizer.merge(doc.ents[i])
print("After:", [token.text for token in doc])

Before: ['Joe Biden', 'married', 'Jill Tracy', 'Jacobs', ',', 'who', 'was', 'educated', 'at', 'the', 'University', 'of', 'Delaware']
After: ['Joe Biden', 'married', 'Jill Tracy', 'Jacobs', ',', 'who', 'was', 'educated', 'at', 'the', 'University', 'of', 'Delaware']


## Named Entity Linking (NEL)

Named Entity Linking (NEL) is the task to task associate mentions of entities in a text with the existing nodes of a target knowledge graph.

In [207]:
print([ent.kb_id_ for ent in doc.ents])

['http://dbpedia.org/resource/Joe_Biden', 'http://dbpedia.org/resource/Jill_Tracy', 'http://dbpedia.org/resource/Delaware']


## Relation Extraction (RE)

Relation Extraction (RE) is the task of detecting and classifying relationships between entities extracted from text.

In [227]:
# we define some patterns to look for in the sentences

def get_relation(sent):
  aux = []
  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [[{'ENT_TYPE' : 'DBPEDIA_ENT'}],[{'DEP' : 'ROOT'}],[{'DEP' : 'relcl'}],
            [{'POS' : 'VERB' }]]

  matcher.add("matching_1",pattern) 

  matches = matcher(doc)
  for x in matches:
    print(doc[x[1]:x[2]].ents)
    aux.append(doc[x[1]:x[2]].text)

  return(aux)

In [228]:
rels = get_relation("Joe Biden married Jill Tracy Jacobs, who was educated at the University of Delaware")
print(rels)

[]
[]
[]
[]
[]
[]
[Delaware]
['Joe', 'Biden', 'married', 'Jill', 'Tracy', 'educated', 'Delaware']


In [160]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS
from rdflib import Namespace
g = Graph()

for ent in doc.ents:
    # add the label of the entity
    g.add((URIRef(ent.kb_id_), RDFS.label, Literal(ent.text)))
    prev = URIRef(ent.kb_id_)
    g.add((URIRef(ent.kb_id_), URIRef("https://dbpedia.org/ontology/"+rels[0]), URIRef(ent2.kb_id_)))


[]


IndexError: list index out of range

In [151]:
for s, p, o in g:
    print(s,p,o)

http://dbpedia.org/resource/Jill_Tracy https://dbpedia.org/ontology/m http://dbpedia.org/resource/Joe_Biden
http://dbpedia.org/resource/Joe_Biden https://dbpedia.org/ontology/married http://dbpedia.org/resource/Jill_Tracy
http://dbpedia.org/resource/Joe_Biden http://www.w3.org/2000/01/rdf-schema#label Joe Biden
http://dbpedia.org/resource/Jill_Tracy http://www.w3.org/2000/01/rdf-schema#label Jill Tracy
http://dbpedia.org/resource/Delaware http://www.w3.org/2000/01/rdf-schema#label Delaware
http://dbpedia.org/resource/Delaware https://dbpedia.org/ontology/m http://dbpedia.org/resource/Joe_Biden
http://dbpedia.org/resource/Delaware https://dbpedia.org/ontology/m http://dbpedia.org/resource/Jill_Tracy
http://dbpedia.org/resource/Jill_Tracy https://dbpedia.org/ontology/m http://dbpedia.org/resource/Delaware
http://dbpedia.org/resource/Joe_Biden https://dbpedia.org/ontology/m http://dbpedia.org/resource/Delaware
